# Name: Anon Hossain

# **ID: 212-35-732**

# **Lab Report Implimentation With Classification Dataset**


---



---



# **Import Libraries**

In [4]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# **Read The Dataset**

In [2]:
df = pd.read_csv('/content/train.csv')


# **Basic Information of dataset**

In [3]:
# Display the first few rows
print("Head of the DataFrame:")
print(df.head(), "\n\n")

# Display the column names
print("Columns in the DataFrame:")
print(df.columns, "\n\n")

# Display DataFrame info
print("Info about the DataFrame:")
print(df.info(),"\n\n")

#Display Description of data
print("Description of the DataFrame:")
print(df.describe(),"\n\n")

#Display total null values
print("Null Values in the DataFrame:")
print(df.isnull().sum(),"\n\n")

#Display Data types
print("Data Types in the DataFrame:")
print(df.dtypes, "\n\n")

#Display Shape
print("Shape of the DataFrame:")
print(df.shape, "\n\n")


Head of the DataFrame:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0

# **Null Value Fillup**

In [5]:
# Assume df is your DataFrame

# First, ensure 'Embarked' is encoded because IterativeImputer works with numeric data
df['Embarked'] = df['Embarked'].astype('category').cat.codes.replace(-1, np.nan)  # Replace -1 (missing) with np.nan

# Select columns you want to impute
cols_to_impute = ['Age', 'Embarked']

# Initialize the imputer
imputer = IterativeImputer(random_state=0)

# Apply imputer to the selected columns
df[cols_to_impute] = imputer.fit_transform(df[cols_to_impute])

# If you want to convert 'Embarked' back to original category
# Recreate the mapping
original_categories = df['Embarked'].dropna().astype('category').cat.categories
df['Embarked'] = pd.Categorical.from_codes(df['Embarked'].round().astype(int), original_categories)

# Final check for missing values
print(df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


# **Build 2 Random Forests using Xgboost or LightGBM library**

In [6]:
target_column = 'Survived'
X = df.drop(target_column, axis=1)
y = df[target_column]


X = pd.get_dummies(X)

X.columns = X.columns.str.replace('[^a-zA-Z0-9_]', '_', regex=True)


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. XGBoost Model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42) # or 'multi:softmax' for multi-class
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_pred)
print(f"XGBoost Accuracy: {xgb_accuracy}")


# 2. LightGBM Model
lgb_model = lgb.LGBMClassifier(objective='binary', random_state=42) # or 'multiclass' for multi-class
lgb_model.fit(X_train, y_train)
lgb_pred = lgb_model.predict(X_test)
lgb_accuracy = accuracy_score(y_test, lgb_pred)
print(f"LightGBM Accuracy: {lgb_accuracy}")

XGBoost Accuracy: 0.7932960893854749
[LightGBM] [Info] Number of positive: 268, number of negative: 444
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000790 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 440
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.376404 -> initscore=-0.504838
[LightGBM] [Info] Start training from score -0.504838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

# **Finding the best set of parameters using GridSearchCV.**

In [7]:
xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

lgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 50, 100],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# 1. XGBoost with GridSearchCV
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid, cv=3, scoring='accuracy', n_jobs=-1) #n_jobs=-1 use all processors
xgb_grid_search.fit(X_train, y_train)

print("Best XGBoost parameters:", xgb_grid_search.best_params_)
xgb_pred = xgb_grid_search.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_pred)

print(f"XGBoost Accuracy (with GridSearchCV): {xgb_accuracy}")

# 2. LightGBM with GridSearchCV
lgb_model = lgb.LGBMClassifier(objective='binary', random_state=42)
lgb_grid_search = GridSearchCV(estimator=lgb_model, param_grid=lgb_param_grid, cv=3, scoring='accuracy', n_jobs=-1)
lgb_grid_search.fit(X_train, y_train)

print("Best LightGBM parameters:", lgb_grid_search.best_params_)
lgb_pred = lgb_grid_search.predict(X_test)
lgb_accuracy = accuracy_score(y_test, lgb_pred)

print(f"LightGBM Accuracy (with GridSearchCV): {lgb_accuracy}")


Best XGBoost parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
XGBoost Accuracy (with GridSearchCV): 0.7932960893854749
[LightGBM] [Info] Number of positive: 268, number of negative: 444
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000177 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 440
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.376404 -> initscore=-0.504838
[LightGBM] [Info] Start training from score -0.504838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

# **Model Evaluation**

In [8]:
print("\nXGBoost Model Performance:")
print(classification_report(y_test, xgb_pred)) # change y_pred_xgb to xgb_pred

print("\nLightGBM Model Performance:")
print(classification_report(y_test, lgb_pred)) # change y_pred_lgb to lgb_pred


XGBoost Model Performance:
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       105
           1       0.79      0.68      0.73        74

    accuracy                           0.79       179
   macro avg       0.79      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179


LightGBM Model Performance:
              precision    recall  f1-score   support

           0       0.83      0.85      0.84       105
           1       0.78      0.76      0.77        74

    accuracy                           0.81       179
   macro avg       0.80      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179



# **Cross-validation accuracy**

In [9]:
xgb_cv_score = cross_val_score(xgb_grid_search.best_estimator_, X, y, cv=5, scoring='accuracy') # changing grid_xgb to xgb_grid_search
lgb_cv_score = cross_val_score(lgb_grid_search.best_estimator_, X, y, cv=5, scoring='accuracy') # changing grid_lgb to lgb_grid_search


print("\nXGBoost Cross-Validation Accuracy: {:.4f} ± {:.4f}".format(np.mean(xgb_cv_score), np.std(xgb_cv_score)))
print("LightGBM Cross-Validation Accuracy: {:.4f} ± {:.4f}".format(np.mean(lgb_cv_score), np.std(lgb_cv_score)))

[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000156 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 448
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383427 -> initscore=-0.475028
[LightGBM] [Info] Start training from score -0.475028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

# **Lab Report Implimentation With Regression Dataset**


---



---

# **Import Libraries**

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.metrics import classification_report
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# **Load The Dataset**

In [12]:
df = pd.read_csv('/content/housing.csv')
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

# **Basic EDA of dataset**

In [13]:
#Display Shape
print("Shape of the DataFrame:")
print(df.shape, "\n\n")

# Display the first few rows
print("Head of the DataFrame:")
print(df.head(), "\n\n")

# Display the column names
print("Columns in the DataFrame:")
print(df.columns, "\n\n")

# Display DataFrame info
print("Info about the DataFrame:")
print(df.info(),"\n\n")

#Display Description of data
print("Description of the DataFrame:")
print(df.describe(),"\n\n")

#Display total null values
print("Null Values in the DataFrame:")
print(df.isnull().sum(),"\n\n")

#Display Data types
print("Data Types in the DataFrame:")
print(df.dtypes, "\n\n")


Shape of the DataFrame:
(20640, 10) 


Head of the DataFrame:
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY   


Columns in

# **Fill the null value of dataset**

In [15]:
# Select all numeric columns including 'total_bedrooms'
numeric_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
                'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']

# Apply Iterative Imputer
imputer = IterativeImputer(random_state=0)
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Check if nulls are gone from total_bedrooms
print(df['total_bedrooms'].isnull().sum())
df.isnull().sum()

0


,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
median_house_value,0
ocean_proximity,0


# **Build 2 Random Forests using Xgboost or LightGBM library**

In [16]:
df = pd.get_dummies(df, drop_first=True)


X = df.drop('median_house_value', axis=1)
y = df['median_house_value']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



params1 = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}


lgb_train1 = lgb.Dataset(X_train, y_train)
lgb_eval1 = lgb.Dataset(X_test, y_test, reference=lgb_train1)


early_stopping = lgb.early_stopping(stopping_rounds=50, verbose=False)

gbm1 = lgb.train(params1,
                lgb_train1,
                num_boost_round=200,
                valid_sets=[lgb_eval1],
                callbacks=[early_stopping])


y_pred1 = gbm1.predict(X_test, num_iteration=gbm1.best_iteration)
rmse1 = np.sqrt(mean_squared_error(y_test, y_pred1))
print(f"LightGBM Model 1 RMSE: {rmse1}")


params2 = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'dart',
    'num_leaves': 50,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7
}

lgb_train2 = lgb.Dataset(X_train, y_train)
lgb_eval2 = lgb.Dataset(X_test, y_test, reference=lgb_train2)
# Define the early stopping callback
early_stopping = lgb.early_stopping(stopping_rounds=50)


gbm2 = lgb.train(params2,
                 lgb_train2,
                 num_boost_round=200,
                 valid_sets=lgb_eval2,
                 callbacks=[early_stopping])

y_pred2 = gbm2.predict(X_test, num_iteration=gbm2.best_iteration)
rmse2 = np.sqrt(mean_squared_error(y_test, y_pred2))
print(f"LightGBM Model 2 RMSE: {rmse2}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001766 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1844
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 11
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 207194.693738
LightGBM Model 1 RMSE: 47047.792910472555
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004608 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1844
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 11
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[L

/usr/local/lib/python3.11/dist-packages/lightgbm/callback.py:329: UserWarning: Early stopping is not available in dart mode
  _log_warning("Early stopping is not available in dart mode")


LightGBM Model 2 RMSE: 48168.14072747344


# **Find the best set of parameters using GridSearchCV.**

In [17]:
param_grid = {
    'boosting_type': ['gbdt', 'dart'],
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.05, 0.1, 0.2],
    'feature_fraction': [0.8, 0.9],
    'bagging_fraction': [0.7, 0.8, 0.9]
}

# Create the LightGBM model
gbm = lgb.LGBMRegressor(objective='regression', metric='rmse')

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=2, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)


print(f"Best parameters: {grid_search.best_params_}")
print(f"Best RMSE: {np.sqrt(-grid_search.best_score_)}")


y_pred = grid_search.predict(X_test)

# Evaluate the best model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Test RMSE: {rmse}")


Fitting 3 folds for each of 108 candidates, totalling 324 fits
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001683 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1844
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 11
[LightGBM] [Info] Start training from score 207194.693738
Best parameters: {'bagging_f